A user table ("takehome_users") with data on 12,000 users who signed up for the product in the last two years.  This table includes: 
* **name**: the user's name
* **object_id**: the user's id
* **email**: email address 
* **creation_source**: how their account was created. This takes on one of 5 values:
 * PERSONAL_PROJECTS: invited to join another user's personal workspace
 * GUEST_INVITE: invited to an organization as a guest (limited permissions)
 * ORG_INVITE: invited to an organization (as a full member)
 * SIGNUP: signed up via asana.com
 * SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login id)
* **creation_time**: when they created their account
* **last_session_creation_time**: unix timestamp of last login
* CAT**opted_in_to_mailing_list**: whether they have opted into receiving marketing emails
* CAT**enabled_for_marketing_drip**: whether they are on the regular marketing email drip
* CAT**org_id**: the organization (group of users) they belong to
invited_by_user_id: which user invited them to join (if applicable).

Started by 

1. Got the adopted users data set as: 

$ mysql -u interview --password=interview --database=interview -P 3306 -h data-challenge-9x.cswh4gchpi8n.us-east-1.rds.amazonaws.com < adopted_users.sql | tr '\t' ','  >  adopted_users.csv

2. Created the "size of org when user joined" data set as:

$ mysql -u interview --password=interview --database=interview -P 3306 -h data-challenge-9x.cswh4gchpi8n.us-east-1.rds.amazonaws.com < size_org_joined.sql | tr '\t' ','  >  size_org_joined.csv

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl
import statsmodels.api as sm

#%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
#Put all csvs into Data Frames

users_df = pd.read_csv('takehome_users.csv')
user_engagement_df = pd.read_csv('takehome_user_engagement.csv')
adopted_users_df = pd.read_csv('adopted_users.csv')
size_org_joined_df = pd.read_csv('size_org_joined.csv')

In [6]:
# Make dummies out of the creation_source category variable
creation_source_dummies = pd.get_dummies(users_df['creation_source'], prefix='creation_source')
creation_source_dummies.head()

,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0


In [5]:
# Make another column in users that segments weekday or not (0 = weekend, 1 = weekeday)
temp = users_df['creation_time'].apply(lambda x: pd.to_datetime(x).weekday())
users_df['weekday'] = temp.apply(lambda x: 1 if x < 5 else 0)

In [7]:
# Join 'cols to keep' on everything but the first series of dummies for creation_source
data = users_df.join(creation_source_dummies)
# Join the size of org when joined by the object_id
data = pd.merge(data, size_org_joined_df, on='object_id')
# Finally  add the dependent variable, adopted_users
data = pd.merge(adopted_users_df, data, left_on='all_users', right_on='object_id')

data.describe()

,all_users,adopted,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,weekday,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_size
count,11583.000000,11583.000000,11583.000000,8.503000e+03,11583.000000,11583.000000,11583.00000,6201.000000,11583.000000,11583.000000,11583.000000,11583.000000,11583.000000,11583.000000,11583.000000
mean,5999.542433,0.109816,5999.542433,1.380255e+09,0.247950,0.148752,139.50436,5967.517336,0.723388,0.178883,0.356471,0.175257,0.174307,0.115082,28.028231
std,3465.728531,0.312674,3465.728531,1.873177e+07,0.431841,0.355860,123.52896,3384.678441,0.447342,0.383271,0.478977,0.380203,0.379390,0.319135,40.624810
min,1.000000,0.000000,1.000000,1.338801e+09,0.000000,0.000000,0.00000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3005.500000,0.000000,3005.500000,1.364862e+09,0.000000,0.000000,28.00000,3037.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000
50%,5991.000000,0.000000,5991.000000,1.383763e+09,0.000000,0.000000,105.00000,5952.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000
75%,9005.500000,0.000000,9005.500000,1.398606e+09,0.000000,0.000000,234.00000,8830.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,29.000000
max,12000.000000,1.000000,12000.000000,1.402067e+09,1.000000,1.000000,416.00000,11999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,318.000000


In [10]:
data['intercept'] = 1

ind_var_cols = [
    'opted_in_to_mailing_list',
    'enabled_for_marketing_drip',
    'creation_source_GUEST_INVITE', 
    'creation_source_ORG_INVITE', 
    'creation_source_PERSONAL_PROJECTS',
    #'creation_source_SIGNUP', Removed first category dummy for basecase
    'creation_source_SIGNUP_GOOGLE_AUTH', 
    'org_size',
    'weekday',
    'intercept'
    ]
dep_var_col = 'adopted'

In [11]:
# Now to actually run the regession
logit = sm.Logit(data[dep_var_col], data[ind_var_cols])

# fit the model
result = logit.fit()

result.summary()

Optimization terminated successfully.
         Current function value: 0.337745
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                adopted   No. Observations:                11583
Model:                          Logit   Df Residuals:                    11574
Method:                           MLE   Df Model:                            8
Date:                Wed, 25 May 2016   Pseudo R-squ.:                 0.02423
Time:                        11:02:41   Log-Likelihood:                -3912.1
converged:                       True   LL-Null:                       -4009.2
                                        LLR p-value:                 1.032e-37
======================================================================================================
                                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------
opted_in_to_mailing_list               0.0451      0.079      0.571      0.568        -0.110     0.200
enabled_for_marketing_drip            -0.0488      0.097     -0.506      0.613        -0.238     0.140
creation_source_GUEST_INVITE           0.1388      0.094      1.474      0.140        -0.046     0.323
creation_source_ORG_INVITE            -0.1297      0.085     -1.522      0.128        -0.297     0.037
creation_source_PERSONAL_PROJECTS     -0.7129      0.114     -6.248      0.000        -0.937    -0.489
creation_source_SIGNUP_GOOGLE_AUTH     0.1118      0.106      1.051      0.293        -0.097     0.320
org_size                              -0.0119      0.001     -8.887      0.000        -0.014    -0.009
weekday                               -0.0749      0.066     -1.131      0.258        -0.205     0.055
intercept                             -1.6711      0.088    -18.889      0.000        -1.845    -1.498
======================================================================================================
"""

In [50]:
odds = 1/(1+np.exp(-result.params))
print odds

opted_in_to_mailing_list              0.511283
enabled_for_marketing_drip            0.487601
creation_source_GUEST_INVITE          0.534444
creation_source_ORG_INVITE            0.467313
creation_source_PERSONAL_PROJECTS     0.328868
creation_source_SIGNUP_GOOGLE_AUTH    0.527870
org_size                              0.497034
intercept                             0.151301
dtype: float64


In [44]:
result.df_resid

11575.0

From the above, the only significant variables (at conf. 95%) are:

	
opted_in_to_mailing_list	0.0451	0.079	0.572	0.568	-0.110 0.200
enabled_for_marketing_drip	-0.0496	0.097	-0.514	0.608	-0.239 0.140
creation_source_ORG_INVITE	-0.2689	0.082	-3.282	0.001	-0.430 -0.108
creation_source_PERSONAL_PROJECTS	-0.8513	0.112	-7.621	0.000	-1.070 -0.632
creation_source_SIGNUP	-0.1380	0.094	-1.466	0.143	-0.323 0.047
creation_source_SIGNUP_GOOGLE_AUTH	-0.0264	0.104	-0.254	0.799	-0.230 0.177
org_size	-0.0119	0.001	-8.880	0.000	-0.014 -0.009
intercept	-1.5864	0.072	-22.167	0.000	-1.727 -1.446



In [6]:
#users_df['weekend'] = users_df['creation_time'].apply(lambda x: x.weekday())

type(users_df['creation_time'][1])

str

In [25]:
users_df[['creation_time', 'weekday']].head()

,creation_time,weekday
0,2014-04-22 03:53:30,1
1,2013-11-15 03:45:04,1
2,2013-03-19 23:14:52,1
3,2013-05-21 08:09:28,1
4,2013-01-17 10:14:20,1
